John-Henry Pezzuto - PS #5

In [10]:
#packages
import numpy as np
import scipy.optimize as opt
import scipy.stats as sts
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import scipy.integrate as intgr
import numpy.linalg as lin
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from pandas import Series, DataFrame


#logistic regression packages
import sklearn as sk
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn import metrics 
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import classification_report
from pylab import rcParams

import seaborn as sb



plt.style.use("ggplot")
%matplotlib inline

#data

strongdrink = pd.read_csv("strongdrink.txt")
inc_sim = np.linspace(0.01, 150000, 200) #fake data.

## (a) Use a multinomial logistic regression model

In [13]:
X = strongdrink[['alco', 'malic', 'tot_phen', 'color_int']]
y = strongdrink['cultivar']


X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size = 0.25,
random_state=20)


LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)
y_pred = LogReg.predict(X_test)

print(classification_report(y_test, y_pred))
# Note that the squared doesn't matter in a Logistic model

print('Validation set MSE = ', mean_squared_error(y_test, y_pred))

             precision    recall  f1-score   support

          1       0.92      0.85      0.88        13
          2       0.95      0.95      0.95        21
          3       0.91      1.00      0.95        10

avg / total       0.93      0.93      0.93        44

Validation set MSE =  0.136363636364


## (b) Perform a leave-one-out cross validation (LOOCV)

In [33]:
#strongdrink
#strongdrink.ix[:, 1].values
#'alco', 'malic', 'tot_phen', 'color_int'

In [29]:
# Define loo as a leave-one-out object, then
# split it into N different partitions

# Note that the LeaveOneOut() function does not work
# well with pandas DataFrames
Xvars = strongdrink.ix[:, (1, 2, 6, 10)].values
yvals = strongdrink.ix[:, 0].values
N_loo = Xvars.shape[0]
loo = LeaveOneOut()
loo.get_n_splits(Xvars)
MSE_vec = np.zeros(N_loo)

# This loop will take 20 or 30 seconds
for train_index, test_index in loo.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec[test_index] = (y_test - y_pred) ** 2
    #print('MSE for test set', test_index, ' is', MSE_vec[test_index])

MSE_loo = MSE_vec.mean()
MSE_loo_std = MSE_vec.std()
print('test estimate MSE loocv=', MSE_loo,
      ', test estimate MSE standard err=', MSE_loo_std)

test estimate MSE loocv= 0.1875 , test estimate MSE standard err= 0.597402045984


## (c) Perform a k-fold cross validation

In [32]:
k=3
kf = KFold(n_splits=k, shuffle=True, random_state=10)
kf.get_n_splits(Xvars)
MSE_vec_kf = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('k index=', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('test estimate MSE k-fold=', MSE_kf,
      'test estimate MSE standard err=', MSE_kf_std)

k index= 0
MSE for test set 0  is 0.28813559322
k index= 1
MSE for test set 1  is 0.271186440678
k index= 2
MSE for test set 2  is 0.051724137931
test estimate MSE k-fold= 0.203682057276 test estimate MSE standard err= 0.107673040209


# 2. Spline and kernel density interpolation (4 points)